In [1]:
import pandas as pd
import itertools
import numpy as np
import openpyxl

In [2]:
fuel_dict = {'MGO': 1, 'LNG': 2, 'Methanol': 3, 'Ammonia': 4, 'B30': 5,  'EL40': 6}

t2_values = range(2,6)
w_values = range(1,7)
scenario_range = range(1,28)
period_range = [2, 3, 4, 5]
# r_values = range(1,1819)

fuel_price_csv = 'fuel_price2_27s.csv'

In [3]:
scenario_developments = [
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2362, 0.2502, 0.2641, 0.2780], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHH 1
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2272, 0.2321, 0.2370, 0.2420], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHM 2
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2182, 0.2141, 0.2100, 0.2059], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHL, 3
    
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2362, 0.2502, 0.2641, 0.2780],  'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HMH, 4
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2272, 0.2321, 0.2370, 0.2420],  'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HMM 5
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2182, 0.2141, 0.2100, 0.2059],  'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HML 6
    
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2362, 0.2502, 0.2641, 0.2780],  'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLH 7
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2272, 0.2321, 0.2370, 0.2420],  'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLM 8
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2182, 0.2141, 0.2100, 0.2059],  'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLL 9
    # 9
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2289, 0.2355, 0.2421, 0.2488],  'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHH 10
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2199, 0.2175, 0.2151, 0.2127],  'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHM  11
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2109, 0.1995, 0.1881, 0.1766],  'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHL 12
    
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2289, 0.2355, 0.2421, 0.2488],  'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MMH 13
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2199, 0.2175, 0.2151, 0.2127],  'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MMM 14
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2109, 0.1995, 0.1881, 0.1766],  'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MML 15
    
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2289, 0.2355, 0.2421, 0.2488], 'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLH 16
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2199, 0.2175, 0.2151, 0.2127], 'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLM 17
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2109, 0.1995, 0.1881, 0.1766],  'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLL 18
    # 18
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHH 19
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHM 20
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHL 21
    
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LMH 22
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LMM 23
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LML 24
    
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLH 25
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLM 26
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLL 27
]

print(scenario_developments[1]['MGO'])

[0.173, 0.1746, 0.1761, 0.1777]


In [4]:
fuel_prices_data = []

# Iterate through the scenario_developments dictionary and construct the data_list
for i, scenario in enumerate(scenario_developments, start=1):
    for fuel, values in scenario.items():
        time_period = 2
        for j, value in enumerate(values):
            fuel_prices_data.append({
                'F': fuel_dict[fuel],
                'T': time_period,
                'S': i,
                'Value': value
            })
            time_period +=1

print(fuel_prices_data)
fuel_price_df = pd.DataFrame(fuel_prices_data)

print(fuel_price_df)

KeyError: 'B20'

In [ ]:

#                 
fuel_price_df.to_csv(fuel_price_csv, index=False)

print(fuel_price_df)

In [ ]:
gamle_scenario_developments = [
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2362, 0.2502, 0.2641, 0.2780], 'B20':[0.2151, 0.2250, 0.2348, 0.2446] , 'B15': [0.2046, 0.2124, 0.2201, 0.2279] , 'B7': [0.1788, 0.1743, 0.1698, 0.1653], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHH 1
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2272, 0.2321, 0.2370, 0.2420], 'B20':[0.2091, 0.2129, 0.2167, 0.2205] , 'B15': [0.2001, 0.2033, 0.2066, 0.2098] , 'B7': [0.1811, 0.1790, 0.1769, 0.1748], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHM 2
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2182, 0.2141, 0.2100, 0.2059], 'B20':[0.2031, 0.2009, 0.1987, 0.1965] , 'B15': [0.1956, 0.1943, 0.1931, 0.1918] , 'B7': [0.1835, 0.1838, 0.1840, 0.1843], 'EL40': [0.2530, 0.2403, 0.2275, 0.2147] }, # HHL, 3
    
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2362, 0.2502, 0.2641, 0.2780], 'B20':[0.2151, 0.2250, 0.2348, 0.2446] , 'B15': [0.2046, 0.2124, 0.2201, 0.2279] , 'B7': [0.1788, 0.1743, 0.1698, 0.1653], 'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HMH, 4
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2272, 0.2321, 0.2370, 0.2420], 'B20':[0.2091, 0.2129, 0.2167, 0.2205] , 'B15': [0.2001, 0.2033, 0.2066, 0.2098] , 'B7': [0.1811, 0.1790, 0.1769, 0.1748], 'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HMM 5
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2182, 0.2141, 0.2100, 0.2059], 'B20':[0.2031, 0.2009, 0.1987, 0.1965] , 'B15': [0.1956, 0.1943, 0.1931, 0.1918] , 'B7': [0.1835, 0.1838, 0.1840, 0.1843], 'EL40': [0.2466, 0.2274, 0.2081, 0.1889] }, # HML 6
    
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2362, 0.2502, 0.2641, 0.2780], 'B20':[0.2151, 0.2250, 0.2348, 0.2446] , 'B15': [0.2046, 0.2124, 0.2201, 0.2279] , 'B7': [0.1788, 0.1743, 0.1698, 0.1653], 'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLH 7
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2272, 0.2321, 0.2370, 0.2420], 'B20':[0.2091, 0.2129, 0.2167, 0.2205] , 'B15': [0.2001, 0.2033, 0.2066, 0.2098] , 'B7': [0.1811, 0.1790, 0.1769, 0.1748], 'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLM 8
    {'MGO': [0.1730, 0.1746, 0.1761, 0.1777], 'LNG': [0.1586, 0.1545, 0.1504, 0.1463], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2182, 0.2141, 0.2100, 0.2059], 'B20':[0.2031, 0.2009, 0.1987, 0.1965] , 'B15': [0.1956, 0.1943, 0.1931, 0.1918] , 'B7': [0.1835, 0.1838, 0.1840, 0.1843], 'EL40': [0.2401, 0.2145, 0.1888, 0.1631] }, # HLL 9
    # 9
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2289, 0.2355, 0.2421, 0.2488], 'B20':[0.2068, 0.2082, 0.2097, 0.2111] , 'B15': [0.1957, 0.1946, 0.1935, 0.1923] , 'B7': [0.1780, 0.1728, 0.1675, 0.1622], 'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHH 10
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2199, 0.2175, 0.2151, 0.2127], 'B20':[0.2008, 0.1962, 0.1917, 0.1871] , 'B15': [0.1912, 0.1856, 0.1799, 0.1743] , 'B7': [0.1759, 0.1685, 0.1612, 0.1538], 'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHM  11
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2109, 0.1995, 0.1881, 0.1766], 'B20':[0.1948, 0.1842, 0.1736, 0.1631] , 'B15': [0.1867, 0.1766, 0.1664, 0.1563] , 'B7': [0.1738, 0.1643, 0.1549, 0.1454], 'EL40': [0.2457, 0.2256, 0.2055, 0.1855] }, # MHL 12
    
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2289, 0.2355, 0.2421, 0.2488], 'B20':[0.2068, 0.2082, 0.2097, 0.2111] , 'B15': [0.1957, 0.1946, 0.1935, 0.1923] , 'B7': [0.1780, 0.1728, 0.1675, 0.1622], 'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MMH 13
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2199, 0.2175, 0.2151, 0.2127], 'B20':[0.2008, 0.1962, 0.1917, 0.1871] , 'B15': [0.1912, 0.1856, 0.1799, 0.1743] , 'B7': [0.1759, 0.1685, 0.1612, 0.1538], 'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MMM 14
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2109, 0.1995, 0.1881, 0.1766], 'B20':[0.1948, 0.1842, 0.1736, 0.1631] , 'B15': [0.1867, 0.1766, 0.1664, 0.1563] , 'B7': [0.1738, 0.1643, 0.1549, 0.1454], 'EL40': [0.2393, 0.2127, 0.1862, 0.1597] }, # MML 15
    
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2289, 0.2355, 0.2421, 0.2488], 'B20':[0.2068, 0.2082, 0.2097, 0.2111] , 'B15': [0.1957, 0.1946, 0.1935, 0.1923] , 'B7': [0.1780, 0.1728, 0.1675, 0.1622], 'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLH 16
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2199, 0.2175, 0.2151, 0.2127], 'B20':[0.2008, 0.1962, 0.1917, 0.1871] , 'B15': [0.1912, 0.1856, 0.1799, 0.1743] , 'B7': [0.1759, 0.1685, 0.1612, 0.1538], 'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLM 17
    {'MGO': [0.1625, 0.1536, 0.1448, 0.1359], 'LNG': [0.1495, 0.1362, 0.1230, 0.1098], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2109, 0.1995, 0.1881, 0.1766], 'B20':[0.1948, 0.1842, 0.1736, 0.1631] , 'B15': [0.1867, 0.1766, 0.1664, 0.1563] , 'B7': [0.1738, 0.1643, 0.1549, 0.1454], 'EL40': [0.2328, 0.1998, 0.1668, 0.1339] }, # MLL 18
    # 18
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'B20':[0.1984, 0.1915, 0.1846, 0.1777] , 'B15': [0.1868, 0.1768, 0.1668, 0.1568] , 'B7': [0.1683, 0.1533, 0.1383, 0.1234], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHH 19
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'B20':[0.1924, 0.1795, 0.1666, 0.1537] , 'B15': [0.1823, 0.1678, 0.1553, 0.1388] , 'B7': [0.1662, 0.1491, 0.1320, 0.1149], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHM 20
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.5085, 0.5028, 0.4970, 0.4912], 'Ammonia': [0.3272, 0.3261, 0.3251, 0.3240], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'B20':[0.1864, 0.1675, 0.1485, 0.1296] , 'B15': [0.1778, 0.1588, 0.1398, 0.1207] , 'B7': [0.1641, 0.1449, 0.1257, 0.1065], 'EL40': [0.2384, 0.2110, 0.1836, 0.1562] }, # LHL 21
    
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'B20':[0.1984, 0.1915, 0.1846, 0.1777] , 'B15': [0.1868, 0.1768, 0.1668, 0.1568] , 'B7': [0.1683, 0.1533, 0.1383, 0.1234], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LMH 22
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'B20':[0.1924, 0.1795, 0.1666, 0.1537] , 'B15': [0.1823, 0.1678, 0.1553, 0.1388] , 'B7': [0.1662, 0.1491, 0.1320, 0.1149], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LMM 23
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4798, 0.4453, 0.4108, 0.3763], 'Ammonia': [0.3089, 0.2895, 0.2702, 0.2509], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'B20':[0.1864, 0.1675, 0.1485, 0.1296] , 'B15': [0.1778, 0.1588, 0.1398, 0.1207] , 'B7': [0.1641, 0.1449, 0.1257, 0.1065], 'EL40': [0.2320, 0.1981, 0.1643, 0.1304] }, # LML 24
    
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2216, 0.2209, 0.2202, 0.2195], 'B20':[0.1984, 0.1915, 0.1846, 0.1777] , 'B15': [0.1868, 0.1768, 0.1668, 0.1568] , 'B7': [0.1683, 0.1533, 0.1383, 0.1234], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLH 25
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2126, 0.2029, 0.1932, 0.1834], 'B20':[0.1924, 0.1795, 0.1666, 0.1537] , 'B15': [0.1823, 0.1678, 0.1553, 0.1388] , 'B7': [0.1662, 0.1491, 0.1320, 0.1149], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLM 26
    {'MGO': [0.1521, 0.1328, 0.1134, 0.0941], 'LNG': [0.1403, 0.1180, 0.0956, 0.0732], 'Methanol': [0.4511, 0.3878, 0.3246, 0.2613], 'Ammonia': [0.2906, 0.2530, 0.2153, 0.1777], 'B30': [0.2036, 0.1848, 0.1661, 0.1474], 'B20':[0.1864, 0.1675, 0.1485, 0.1296] , 'B15': [0.1778, 0.1588, 0.1398, 0.1207] , 'B7': [0.1641, 0.1449, 0.1257, 0.1065], 'EL40': [0.2255, 0.1852, 0.1449, 0.1046] }, # LLL 27
]